In [30]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [33]:
COMPANY = "microsoft-corp"   

In [34]:
url = f'https://www.investing.com/equities/{COMPANY}-financial-summary'
r = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
sheet = soup.find(class_="min-w-0")

In [35]:
ticker = sheet.find("h1").text.split('(')[1].strip().replace(')', '')
current_price = sheet.find(class_="text-5xl/9 font-bold text-[#232526] md:text-[42px] md:leading-[60px]").text
percent_change = sheet.find("span", {'data-test': 'instrument-price-change-percent'}).text
p_e_ratio = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[0].text
price_book_ratio = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[1].text
debt_equity_ratio = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[2].text
return_on_equity = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[3].text
dividend_yield = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[4].text
ebitda_margin = sheet.find_all("span", class_="block text-sm leading-5 rtl:text-right rtl:soft-ltr")[5].text

In [ ]:
print(f'Ticker : {ticker}')
print(f'Current Price : {current_price} {percent_change}')
print(f'P/E Ratio : {p_e_ratio}')
print(f'Price/Book Ratio : {price_book_ratio}')
print(f'Debt/Equity Ratio : {debt_equity_ratio}')
print(f'Return on Equity : {return_on_equity}')
print(f'Dividend Yield : {dividend_yield}')
print(f'EBITDA Margin : {ebitda_margin}')

Ticker : MSFT
Current Price : 456.16 (-0.59%)
P/E Ratio : 35.21
Price/Book Ratio : 10.05
Debt/Equity Ratio : 32.63%
Return on Equity : 33.61%
Dividend Yield : 0.73%
EBITDA Margin : 129.43B


In [ ]:
table = soup.find(class_="w-full text-xs leading-4 indicators-table_table__lfa5n indicators-table_table--freeze-column__MonUl")
thead = table.find('thead')
years=[]
for th in thead.find_all('th',class_="indicators-table_cell__m6pYi min-w-[100px] border-b border-[#D9DCDF] py-2.5 pr-3 text-right !text-xs font-normal rtl:text-right"):
    year=th.find('div', class_='font-semibold').text
    years.append(year)

In [87]:
years=years[5:]

In [ ]:
statements_df= pd.DataFrame(columns='Total_Revenues, Gross_Profit, Operating_Income, Net_Income, Total_Assets, Total_Liabilities, Total_Equity, Cash and Cash Equivalents, Total Debt'.split(', '), index=years)

In [89]:
statements_df

,Total Revenues,Cost of Goods Sold,Gross Profit,Operating Income,Net Income,Total Assets,Total Liabilities,Shareholder Equity,Cash and Cash Equivalents,Total Debt
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
year_div = table.find('div', class_='font-semibold')

year = None
if year_div:
    year = year_div.text.strip()
    print(f"Extracted year: '{year}'")
else:
    print("Year div not found.")

# If you want to convert it to an integer:
if year:
    try:
        year_int = int(year)
        print(f"Extracted year (integer): {year_int}")
    except ValueError:
        print(f"Could not convert '{year}' to an integer.")

Year div not found.
